In [ ]:
import time

import cv2
import numpy as np
from matplotlib import pyplot as plt

from common import coco_part_list
from estimator import TfPoseEstimator
from networks import get_graph_path, model_wh

In [ ]:
fps_time = 0

count = 0

if __name__ == '__main__':
    args = {'camera': 0, 'zoom': 1.0, 'resolution': '256x144', 'model': 'mobilenet_thin', 'show-process': False,
            'magnify': 1.0}

    w, h = model_wh(args['resolution'])
    e = TfPoseEstimator(get_graph_path(args['model']), target_size=(w, h))
    
    cam = cv2.VideoCapture(args['camera'])
        
    try:
        ret_val, image = cam.read()

        while True:
            ret_val, image = cam.read()

            if args['zoom'] < 1.0:
                canvas = np.zeros_like(image)
                img_scaled = cv2.resize(image, None, fx=args['zoom'], fy=args['zoom'], interpolation=cv2.INTER_LINEAR)
                dx = (canvas.shape[1] - img_scaled.shape[1]) // 2
                dy = (canvas.shape[0] - img_scaled.shape[0]) // 2
                canvas[dy:dy + img_scaled.shape[0], dx:dx + img_scaled.shape[1]] = img_scaled
                image = canvas
            elif args['zoom'] > 1.0:
                img_scaled = cv2.resize(image, None, fx=args['zoom'], fy=args['zoom'], interpolation=cv2.INTER_LINEAR)
                dx = (img_scaled.shape[1] - image.shape[1]) // 2
                dy = (img_scaled.shape[0] - image.shape[0]) // 2
                image = img_scaled[dy:image.shape[0], dx:image.shape[1]]

            humans = e.inference(image)
            if humans:
                for _, x in humans[0].body_parts.items():
                    print('(%r, %r): %r, %r' % (x.x, x.y, coco_part_list[x.part_idx], x.score))
                print()
            
            image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

            cv2.putText(image,
                        "FPS: %f" % (1.0 / (time.time() - fps_time)),
                        (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (0, 255, 0), 2)
            image = cv2.resize(image, None, fx=args['magnify'], fy=args['magnify'],
                               interpolation=cv2.INTER_LINEAR)
            cv2.imshow('Pose Estimation', image)
            
            fps_time = time.time()
            if cv2.waitKey(1) == 27:
                break
                
            count += 1
            if count > 50:
                break

    finally:
        cv2.destroyAllWindows()
        cam.release()